In [25]:
from pathlib import Path
import json
import csv
import pandas as pd
from pandas.io.json import json_normalize
from prettytable import PrettyTable

In [26]:
def remove_struct(df):
    L_to_keep = ['0_name', 'epochs', 'learning_rate', 'dropout', 'learning_rate_decay_type']
    for param in df.columns.tolist():
        if param not in L_to_keep:
            df = df.drop(param, axis=1, errors='ignore')
    return df

In [27]:
def config_decode(array_config, path_model, type_):
    # Loading and flattening json file
    path_config = path_model / 'config_network.json'
    model_name = path_model.parts[-1]

    with open(path_config, 'r') as config_file:
        config = json.loads(config_file.read())
        config_flatten = json_normalize(config)
        #config_flatten.rename(columns=lambda x: x[8:], inplace=True)
        config_flatten = remove_struct(config_flatten)
        config_flatten.insert(0,'0_name',model_name)
        # Appending to existing dataframe
        res = array_config.append(config_flatten)
        return res

In [28]:
def describe(write_model):
    models = pd.DataFrame()

    for item_path in Path.cwd().iterdir():
        if item_path.is_dir():
            for data in item_path.iterdir():
                if data.match('*config_network*'):
                    models = config_decode(models, item_path, 'describe')


    models.to_csv('models_description.csv', index=False)

    # Now we display the differences between the dataframes
    t = PrettyTable(models.columns.tolist())
    for index, rows in models.iterrows():
        t.add_row(rows)
    print(t)

In [29]:
describe(True)

/Users/kwu2/miniconda/envs/ads_venv/lib/python3.7/site-packages/ipykernel_launcher.py:8: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  


+-------------------------------------------+--------+---------+---------------+--------------------------+
|                   0_name                  | epochs | dropout | learning_rate | learning_rate_decay_type |
+-------------------------------------------+--------+---------+---------------+--------------------------+
|        CNS_TEM_2021-11-04_10-07-52        |  2000  |   0.75  |     0.001     |        polynomial        |
|        CNS_TEM_2021-11-16_01-25-20        |  300   |   0.75  |     0.001     |        polynomial        |
|        CNS_TEM_2021-12-01_02-57-01        |  2000  |   0.75  |     0.001     |        polynomial        |
|          TEM_2021-06-08_14-58-45          |  600   |   0.75  |      0.01     |        polynomial        |
|        CNS_TEM_2021-12-01_02-28-15        |  2000  |   0.6   |     0.001     |        polynomial        |
|          TEM_2021-10-18_01-44-32          |  600   |   0.75  |      0.01     |        polynomial        |
|               best_CNS_mod